In [307]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime
from dateutil.relativedelta import relativedelta
# set the settingWithCopyWarning in Pandas to None
pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

#### The aim of this notebook is to generate product usage data from scratch. We generate data for 10.000 users and simulate their behavior which can then we used for cohort analysis like creating retention curves. More here: http://josolnik.com/generating_product_usage_data.html

### 1. Defining the main parameters

In [308]:
# number of devices to generate data for
num_device_uuids = 100

# number of months generated for each device
num_months = 20

# starting month of product usage data
start_month = '2016-01-01'

###  2. Generating device uuids and applying them to datetime data

In [309]:
# generating unique identifiers for each device
uuids = pd.Series([str(uuid.uuid4()) for i in range(1,num_device_uuids)])
user_data = pd.DataFrame()
user_data['device_uuid'] = pd.Series(uuids).repeat(num_months)
user_data =user_data.reset_index().drop('index', 1)

# example of two different device uuids (last row of first device, first row of second device)
user_data[19:21]

,device_uuid
19,5113ec00-46fd-44eb-af5c-19c6a95a92e9
20,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6


In [310]:
# defining starting and ending month for generating data
start_month_ts = pd.to_datetime(start_month)
end_month_ts = start_month_ts + relativedelta(months=+num_months - 1)
str(start_month_ts), str(end_month_ts)

('2016-01-01 00:00:00', '2017-08-01 00:00:00')

In [311]:
# months for which we will be generating data
months = pd.Series(pd.date_range(start_month_ts, end_month_ts, freq='MS'))
months

0    2016-01-01
1    2016-02-01
2    2016-03-01
3    2016-04-01
4    2016-05-01
5    2016-06-01
6    2016-07-01
7    2016-08-01
8    2016-09-01
9    2016-10-01
10   2016-11-01
11   2016-12-01
12   2017-01-01
13   2017-02-01
14   2017-03-01
15   2017-04-01
16   2017-05-01
17   2017-06-01
18   2017-07-01
19   2017-08-01
dtype: datetime64[ns]

In [312]:
# adding date column to the user data dataframe
user_data['date'] = pd.concat([months] * num_device_uuids, axis=0).reset_index().drop('index', 1)
user_data.head()

,device_uuid,date
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-01-01
1,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-02-01
2,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-03-01
3,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-04-01
4,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-05-01


### 3. Generating usage features

In [316]:
# defining parameters for usage features (used later)
num_usage_features = 3
feature_usage_ratio = 0.8

### 3.1. Calculating feature ratios

In [317]:
# extracting all the unique device uuids from the main df
device_uuids = pd.Series(user_data['device_uuid'].unique())
user_data_with_usage = pd.DataFrame()
device_uuids[0:5]

0    5113ec00-46fd-44eb-af5c-19c6a95a92e9
1    a002e38e-3a47-4ed1-a252-0d3a2e47c2f6
2    fd0eb87d-c005-403a-a3be-4d3603a65de2
3    e74ecb22-6393-433f-8fde-b95ec7c526cf
4    9ca402a2-8d88-412f-a14b-6c3946f55046
dtype: object

In [318]:
# generate random usage data for each feature and simulate dimishing usage per feature
# (each feature 80% of the previous one)
# example: F1 = 1, F2 = F1*0.8 = 0.8, F3 = F2*0.8 = 0.64

# extract the names for each feature based on the preset number of feature
features = ['feature' + str(i) for i in range(1,num_usage_features+1)]

usage_features_ratio = {}

# set the feature_ratio for the first feature (100%)
cur_feature = 1.0

# calculate the ratio for each feature
for feature in features:
    usage_features_ratio[feature] = float("{0:.1f}".format(cur_feature))
    cur_feature*= 0.8

print("Feature ratios: ", usage_features_ratio)

Feature ratios:  {'feature1': 1.0, 'feature3': 0.6, 'feature2': 0.8}


### 3.2. Assigning cohort groups to devices

##### When did the user start using the product? (How many months to remove from the beginning of devices' dataframe?)

In [319]:
# set the number of cohorts to be half of number of months of data (20 months = 10 cohorts)
num_cohorts = int(num_months / 2) 
num_cohorts

10

In [320]:
# assign cohorts to users randomly (when did the user first used the app?)
cohorts = pd.DataFrame()
cohorts['device_uuid'] = device_uuids
cohorts['cohort_group'] = np.random.randint(low=1, high=num_cohorts, size=num_device_uuids-1)
cohorts.head()

,device_uuid,cohort_group
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,9
1,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,6
2,fd0eb87d-c005-403a-a3be-4d3603a65de2,9
3,e74ecb22-6393-433f-8fde-b95ec7c526cf,8
4,9ca402a2-8d88-412f-a14b-6c3946f55046,9


### 3.3. Adding cohort data

In [321]:
# convert a series of dataframes into a dataframe
# we need this when we get an output as a series for dataframes,
# each dataframe representing data for a specific device
def df_series_to_df(df_series):
    df = pd.concat([df for df in df_series], ignore_index=True)
    return(df)

In [348]:
# how many months to remove from the beginning of a dataframe?
def add_cohorts_to_df(device_uuid):
    
    # 1. slice user_data, extract device data for each user
    device_uuid_data = user_data[user_data['device_uuid'] == device_uuid]
    
    # 2. find cohort group of the device and delete all unnecessary months
    device_uuid_cohort_group = int(cohorts[cohorts['device_uuid'] == device_uuid]['cohort_group'])
    device_uuid_data = device_uuid_data[device_uuid_cohort_group:] 
    
    return(device_uuid_data)

In [349]:
# takes a while to run
user_data_with_cohort_groups_df = df_series_to_df(device_uuids.apply(add_cohorts_to_df))

In [350]:
user_data_with_cohort_groups_df.head()

,device_uuid,date
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-10-01
1,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-11-01
2,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-12-01
3,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2017-01-01
4,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2017-02-01


In [351]:
# average number of months of usage per device_uuid
# down from 20 (deleted the rows where the user stopped using the product - churned)
avg_months_per_device_after_cohorts = round(len(user_data_with_cohort_groups_df) / len(device_uuids))
print("Average number of months per user after simulating cohorts:", avg_months_per_device_after_cohorts, \
     "(down from 20)")

Average number of months per user after simulating cohorts: 15 (down from 20)


### 3.4. Generating product usage data

In [352]:
# reseting the index
user_data_with_cohort_groups_df = user_data_with_cohort_groups_df.reset_index().drop('index', axis=1)

# generate usage for feature1
user_data_with_cohort_groups_df[features[0]] = pd.Series((
        np.random.randint(low=0, high=14, 
        size=len(user_data_with_cohort_groups_df))))


# generate usage for the remaining usage features (feature2 and feature3)
for feature in features[1:]:
        user_data_with_cohort_groups_df[feature] = user_data_with_cohort_groups_df[features[0]] * usage_features_ratio[feature]
        
user_data_with_cohort_groups_df.head()

,device_uuid,date,feature1,feature2,feature3
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-10-01,13,10.4,7.8
1,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-11-01,3,2.4,1.8
2,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-12-01,6,4.8,3.6
3,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2017-01-01,4,3.2,2.4
4,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2017-02-01,13,10.4,7.8


### 3.5. Simulating churn behavior

##### When did the user stop using the product? (How many months to remove from the end of the devices' dataframe?)

#### 2 principles:
    1. Churner stays a churner (after a month of no usage the user doesn't use the product again)
    2. 50 % of users churn in month 1 (about 50% of users have 1 month of data, about 50% have more)

In [353]:
def simulating_churn(device_uuid):

    # set the churner to False
    churner = False
    
    # 50 % chance that the user churns in the first month
    churner_in_m1 = True if int(np.random.randint(low=0, high=2, size=1)) == 0 else False
    
    # get a slice of device_uuid data
    device_uuid_data = user_data_with_cohort_groups_df[user_data_with_cohort_groups_df['device_uuid'] == device_uuid]
    
    
    # if the user churns in month 1 then assign all of the months to 0
    # (except in month 0 when the user first used the product)
    if churner_in_m1:
        device_uuid_data.loc[device_uuid_data.index[1]:, features[0:]] = 0
        
    else:
    # if user didn't churn in the first month
        for index, row in device_uuid_data.iterrows():
        
            # did the user churn in this month?
            if row[features[0]] == 0:
                churner = True   

            # if user churned already set the value of all usage features to 0 for the remaining months
            if churner == True:
                device_uuid_data.loc[index, features[0:]] = 0
                
    # delete months when the user didn't use the product (churn months)
    device_uuid_data = device_uuid_data[device_uuid_data[features[0]] != 0]
    
    return(device_uuid_data)

In [354]:
user_data_with_usage_features_and_churn_df = df_series_to_df(device_uuids.apply(simulating_churn))
user_data_with_usage_features_and_churn_df.head()

,device_uuid,date,feature1,feature2,feature3
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-10-01,13,10.4,7.8
1,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-07-01,10,8.0,6.0
2,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-08-01,6,4.8,3.6
3,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-09-01,7,5.6,4.2
4,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-10-01,12,9.6,7.2


In [355]:
# average number of months of usage per device_uuid
# down from 20 (deleted the rows where the user stopped using the product - churned)
avg_months_per_device_after_churn = round(len(user_data_with_usage_features_and_churn_df) / len(device_uuids))
print("Average number of months per user after simulating churn:", avg_months_per_device_after_churn, \
     "(down from 20)")

Average number of months per user after simulating churn: 4 (down from 20)


### 4. Generating categorical features

In [356]:
# defining categorical parameters
platforms = ['iOS', 'Android']
countries = ['NL', 'AU', 'FR']
user_registered = [False, True]

In [357]:
segmentation_features = {'platform': platforms,
                'user_registered': user_registered,
                'country': countries}
segmentation_features

{'country': ['NL', 'AU', 'FR'],
 'platform': ['iOS', 'Android'],
 'user_registered': [False, True]}

### 4.1. Generating categorical feature weights

In [358]:
# Defining weights for the likelihood of a categorical feature appearing

weights_2 = [0.7, 0.3]
weights_3 = [0.6, 0.3, 0.1]

# If there are two possible variants in a categorical variable (e.g. 'Android' and 'iOS') then the data generated will contain 70% of the first variant and 30% with the second variant
# If there are three possible variants in a categorical variable (e.g. 'NL', 'AU', 'FR') then the data generated will contain 60% of the first variant, 30% with the second variant and 10% with the third variant

### 4.2. Applying the categorical feature weights

In [359]:
def generating_categorical_features(device_uuid):
    
    device_uuid_feature_values_dict = {}
    
    for feature_name, feature_values in segmentation_features.items():

        # generate the random feature value
        # add code if you want to add features with > 3 values
        if len(feature_values) == 2:
            feature_weights = weights_2
        elif len(feature_values) == 3:
            feature_weights = weights_3
        
        # generate the feature value from assigned weights
        feature_value = np.random.choice(feature_values, p=feature_weights)
        
        # make a dict for feature_name
        device_uuid_feature_values_dict[feature_name] = feature_value
        
        
        
    # make a df with segmentation features for the device_uuid
    device_segm_features = pd.DataFrame(list(device_uuid_feature_values_dict.items())).T
    device_segm_features.columns = device_segm_features.iloc[0]
    device_segm_features['device_uuid'] = device_uuid
    device_segm_features = device_segm_features.reindex(device_segm_features.index.drop(0))
    
    return(device_segm_features)

In [360]:
segmentation_features_df = df_series_to_df(device_uuids.apply(generating_categorical_features))
segmentation_features_df.head()

,country,platform,user_registered,device_uuid
0,AU,iOS,True,5113ec00-46fd-44eb-af5c-19c6a95a92e9
1,NL,Android,True,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6
2,AU,iOS,True,fd0eb87d-c005-403a-a3be-4d3603a65de2
3,FR,Android,False,e74ecb22-6393-433f-8fde-b95ec7c526cf
4,NL,Android,False,9ca402a2-8d88-412f-a14b-6c3946f55046


##  5. Merging all features into the final dataframe

In [361]:
def merge_all_features(device_uuid):
    
    # extract device data from the main df (which includes usage data)
    device_uuid_df = user_data_with_usage_features_and_churn_df[user_data_with_usage_features_and_churn_df['device_uuid'] == device_uuid]
    # extract device data for segmentation features data
    device_seg_feature_values_df = segmentation_features_df[segmentation_features_df['device_uuid'] == device_uuid]
    
    # assign the values of segmentation features to the main df
    # works but don't understand it well enough to simply if further
    for seg_feature_name in device_seg_feature_values_df.columns[1:]:
        device_uuid_df[seg_feature_name] = device_seg_feature_values_df[seg_feature_name][int(device_seg_feature_values_df[seg_feature_name].index.values)]

    return(device_uuid_df)

In [362]:
final_df = df_series_to_df(device_uuids.apply(merge_all_features))
final_df.head()

,device_uuid,date,feature1,feature2,feature3,platform,user_registered
0,5113ec00-46fd-44eb-af5c-19c6a95a92e9,2016-10-01,13,10.4,7.8,iOS,True
1,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-07-01,10,8.0,6.0,Android,True
2,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-08-01,6,4.8,3.6,Android,True
3,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-09-01,7,5.6,4.2,Android,True
4,a002e38e-3a47-4ed1-a252-0d3a2e47c2f6,2016-10-01,12,9.6,7.2,Android,True


## 6. Checking simulation output in our final dataframe

#### Did we generate the data that we intended to?

Why? Law of large numbers: The average of the results obtained from a large number of trials should be close to the expected value, and will tend to become closer as more trials are performed) - <a href="https://en.wikipedia.org/wiki/Law_of_large_numbers"> More on that </a> 

### 6.1. Usage features (average per feature)

In [363]:
# checking proportion of usage for usage features
# (each feature 80% of the previous one)
print((final_df['user_registered'].sum() / len(final_df['user_registered'])*100).astype(int), \
      "% users are registered (expected value is 30%)")

34 % users are registered (expected value is 30%)


### 6.2. Segmentation features: Percent of users registered

In [364]:
# checking proportion of usage for usage features
# (each feature 80% of the previous one)
final_df.groupby(['user_registered']).mean()

,feature1,feature2,feature3
user_registered,,,
False,6.771930,5.417544,4.063158
True,7.178808,5.743046,4.307285


In [365]:
print("Feature 1 / Feature 2:", int((final_df[features[1]].mean() / final_df[features[0]].mean())*100), "%", \
     "(expected value is 80%)")

Feature 1 / Feature 2: 80 % (expected value is 80%)


In [366]:
print("Feature 2 / Feature 3:", int((final_df[features[2]].mean() / final_df[features[1]].mean())*100), "%" \
      , "(expected value is 80%)")

Feature 2 / Feature 3: 75 % (expected value is 80%)


### 6.3. Churn simulation

In [367]:
# number of m1 users / number of non-m2 users should be close to 50/50
m1_users = (user_data_with_usage_features_and_churn.groupby('device_uuid')['date'].count() == 1).sum()
non_m1_users = (user_data_with_usage_features_and_churn.groupby('device_uuid')['date'].count() != 1).sum()
print("Number of m1 users / Number of non-m1 users:", m1_users, "/", non_m1_users)

Number of m1 users / Number of non-m1 users: 490 / 440


In [368]:
# checking that 50% of users churn in the first month
m1_users = (final_df.groupby('device_uuid')['date'].count() == 1).sum()
print(((m1_users / len(device_uuids))*100).astype(int), "% of users churn in the first month", \
     ("(expected value is 50%)"))

52 % of users churn in the first month (expected value is 50%)


## 7. Export the final dataframe as a csv

In [369]:
# exporting the final dataframe for later usage
final_df.to_csv('generating_user_behavioral_data.csv')